# Lets build a Deep Feed Forward Neural Network

All dependencies for this notebook is listed in the requirements.txt file. One parent above the nbs directory. This list will keep changing as we add to it so be sure to rerun this line after every git pull

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd gdrive/My Drive/cs271p

In [ ]:
!ls

In [ ]:
!pip install -r requirements.txt

Lets declare our imports

In [ ]:
import numpy as np
import torch
from torch import nn
import math
from pprint import pprint
from tqdm.notebook import tqdm

# Kaggle Datasets

### This is an example of how to download datasets directly from kaggle

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
! kaggle datasets download -d ronitf/heart-disease-uci

In [ ]:
class BasicNeuralNetwork(torch.nn.Module):
    def __init__(self, in_size=2, out_size=1, hidden_size=30):
        super(BasicNeuralNetwork, self).__init__()

        # Set the dimensionality of the network
        self.input_size = in_size
        self.output_size = out_size
        self.hidden_size = hidden_size

        # Initialize our weights
        self._init_weights()

    '''
    Initialize the weights
    '''
    def _init_weights(self):
        # Create an input tensor of shape (2,3)
        self.W_Input = torch.randn(self.input_size, self.hidden_size)

        # Create an output tensor of shape (3, 1)
        self.W_Output = torch.randn(self.hidden_size, self.output_size)

    '''
    Create the forward pass
    '''
    def forward(self, inputs):
        # Lets get the element wise dot product
        self.z = torch.matmul(inputs, self.W_Input)
        # We call the activation
        self.state = self._activation(self.z)
        # Pass it through the hidden layer
        self.z_hidden = torch.matmul(self.state, self.W_Output)
        # Finally activate the output
        output = self._activation(self.z_hidden)

        # Return the output
        return output

    '''
    Backpropagation algorithm implemented
    '''
    def backward(self, inputs, labels, output):
        # What is the error in output
        self.loss = labels - output

        # What is the delta loss based on the derivative
        self.loss_delta = self.loss * self._derivative(output)

        # Get the loss for the existing output weight
        self.z_loss = torch.matmul(self.loss_delta, torch.t(self.W_Output))

        # Compute the delta like before
        self.z_loss_delta = self.z_loss * self._derivative(self.state)

        # Finally propogate this to our existing weight tensors to update
        # the gradient loss
        self.W_Input += torch.matmul(torch.t(inputs), self.z_loss_delta)
        self.W_Output += torch.matmul(torch.t(self.state), self.loss_delta)

    '''
    Here we train the network
    '''
    def train(self, inputs, labels):
        # First we do the foward pass
        outputs = self.forward(inputs)
        # Then we do the backwards pass
        self.backward(inputs, labels, outputs)

    '''
    Here we perform inference
    '''
    def predict(self, inputs):
        pass

    '''
    Here we save the model
    '''
    def save(self):
        pass

    '''
    Our non-linear activation function
    '''
    def _activation(self, s):
        # Lets use sigmoid
        return 1 / (1 + torch.exp(-s))

    '''
    Our derivative function used for backpropagation
    Usually the sigmoid prime
    '''
    def _derivative(self, s):
        # derivative of sigmoid
        return s * (1 - s)

In [ ]:
import pandas as pd
df = pd.read_csv('heart.csv')

In [ ]:
df.head(20)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
sc = MinMaxScaler((-1, 1))

Lets split out dataset between inputs and target

In [ ]:
df.shape
y = df['target']
X = df.drop('target', axis=1)

Lets create a test and train split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

Here we transform features by scaling each feature to a given range.
This estimator scales and translates each feature individually such that it is in the given range on the training set, e.g. between zero and one.

In [ ]:
X_train = sc.fit_transform(X_train)

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float)

In [ ]:
y_train = torch.tensor((y_train.values,))
y_train = y_train.transpose(0,1)

In [ ]:
print(X_train.shape, y_train.shape)

Now we instantiate our neural network

In [ ]:
net = BasicNeuralNetwork(in_size=X_train.shape[1], out_size=1, hidden_size=100)

We train our neural network with 1000 epochs (training loops) and we measure the loss

In [ ]:
for i in tqdm(range(100)):
    outputs = net(X_train)
    loss = torch.mean((y_train - outputs)**2).detach().item()
    tqdm.write("Loss: {}".format(loss))
    net.train(X_train, y_train)

In [ ]:
with torch.no_grad():
  prediction = net(torch.tensor(sc.transform(X_test), dtype=torch.float))
  _, preds_y = torch.max(prediction, 1)

In [ ]:
accuracy_score(y_test, preds_y)

# Excercises

1. Try to initialize the weights with something better. Hint (Xavier Initialization)
2. Add a bias to the forward pass. Recall the affine transform is (inputs . weights) + bias
3. We are missing a learning rate to the backwards pass. See if you can add that in

# How would we rewrite this code using PyTorch built-in methods

PyTorch gives us most of this functionality out of the box. First we can flag all Tensors to use Autograd. You can read more about autograd here: https://pytorch.org/docs/stable/autograd.html

In [ ]:
X = df.drop('thalach', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
# Populate the best
X_train = torch.tensor(sc.fit_transform(X_train), dtype=torch.float, requires_grad=True)
X_test = torch.tensor(sc.transform(X_test), dtype=torch.float)

y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)

This is the first way using the torch.nn.Sequential. In the Sequential model modules will be added to it in the order they are passed in the constructor. This is a quick way to write a small neural network

In [ ]:
import torch.nn as nn
from collections import OrderedDict

model = torch.nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(13, 100)),
    ('relu1', nn.ReLU()),
    ('fc2', nn.Linear(100, 100)),
    ('relu2', nn.ReLU()),
    ('fc3', nn.Linear(100, 2)),
    ('sigmoid', nn.Sigmoid())
]))

What does the architecture of this model look like

In [ ]:
print(model)

Lets register an optimizer and an objective function

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
losses = []

In [ ]:
for epoch in tqdm(range(1000)):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    print("Epoch {}, Loss: {}".format(epoch, loss.item()))

In [ ]:
prediction = model(X_test)
_, preds_y = torch.max(prediction, 1)

In [ ]:
accuracy_score(y_test, preds_y)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses, label="Loss Curve")
plt.legend()
plt.show()

In [ ]:
losses_no_back = []
for epoch in tqdm(range(1000)):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    optimizer.step()
    losses_no_back.append(loss.item())
    print("Epoch {}, Loss: {}".format(epoch, loss.item()))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses_no_back, label="Loss Curve")
plt.legend()
plt.show()

# Is there a better way?



We looked at the torch.nn.Module before. Inherting from this class gives us:

1. More flexibility on how we build our layers
2. Encapsulate our logic into one object
3. Easily swap out optimization functions
4. Easilty swap out cost functions

In [ ]:
class DeepNeuralNetwork(nn.Module):
    def __init__(self, in_size, out_size, hidden_size, layer_depth=4, activation=nn.ReLU):
        super(DeepNeuralNetwork, self).__init__()
        
        self.activation = activation()
        self.in_size = in_size
        self.out_size = out_size
        
        self.fc1 = nn.Linear(in_size, hidden_size)
        self.fcn = nn.ModuleDict({})
        
        for l in range(layer_depth):
            name = 'fc'+str(1+l)
            self.fcn[name] = nn.Linear(hidden_size, hidden_size)
        
        self.out = nn.Linear(hidden_size, out_size)
        
        self.optimizer = torch.optim.Adam(self.parameters())
        self.criterion = nn.CrossEntropyLoss()
        self.loss_tracker = []
    
    def add_layer(index, layer):
        pass
    
    def forward(self, x):
        x = self.activation(self.fc1(x))
        for k, l in self.fcn.items():
            x = self.activation(x)
        x = self.out(x)
        return x
    
    def train(self, inputs, labels, test_inputs=None, test_labels=None, epochs=10) -> None:
        for epoch in tqdm(range(epochs)):
            self.optimizer.zero_grad()
            outputs = self(X_train)
            loss = self.criterion(outputs, y_train)
            loss.backward()
            self.optimizer.step()
            self.loss_tracker.append(loss.item())
            acc = self.accuracy(test_inputs, test_labels)
            tqdm.write("Epoch {}, Loss: {} Acc: {}".format(epoch, loss.item(), acc))

    def accuracy(self, test_inputs, test_labels):
        _, preds_y = torch.max(self(test_inputs), 1)
        return accuracy_score(test_labels, preds_y)
    
    def show_loss(self):
        import matplotlib.pyplot as plt
        plt.plot(self.loss_tracker, label="Loss Curve")
        plt.legend()
        plt.show()
        
    def predict(self, inputs):
        """
        Sets the model to evaluation/inference mode, disabling dropout and
        gradient calculation.
        """
        self.eval()
        return self(inputs)
    
    def summary(self):
        from torchsummary import summary
        summary(self, (1, 1, self.in_size)) 

We create a new instance of the Module we just created

In [ ]:
model = DeepNeuralNetwork(13, 2, 100)

Lets look at what the model summary or architecture looks like. 
We will go into much more depth when look at TensorboardX

In [ ]:
model.summary()

Lets go ahead and train the model

In [ ]:
model.train(X_train, y_train, X_test, y_test, epochs=1000)

Lets get the model accuracy

In [ ]:
model.accuracy(X_test, y_test)

What does the loss curve look like

In [ ]:
model.show_loss()

Lets print out the model summary one more time to see if anything has changed

In [ ]:
model.summary()

The model architecture we built above is sufficient for many problems. 
But lets look at a SOTA model and see how complex it can become

In [ ]:
from PIL import Image
from IPython.display import display
img = Image.open('images/inception_v4.jpeg')
display(img)

# Excercises

1. Try to implement early stopping
2. Implement a different activation function. What worked, what didn't
3. Implement a different loss function. What worked and what didn't
4. Introduce a different dataset and see if you can use the above model to build an accurate model

# References

https://arxiv.org/pdf/1602.07261.pdf

https://pytorch.org/docs/stable/nn.html